In [1]:
# Import necessary modules
import os 
from langchain_core.tools import tool
from langgraph.graph import MessagesState, START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
from display_graph import display_graph
import finnhub
import keyring

# Initialize Finnhub API client
finnhub_client = finnhub.Client(api_key=keyring.get_password('finnhub', 'key_for_windows'))

In [2]:
# Define the tool: querying stock prices using the Finnhub API
@tool
def get_stock_prices(symbol: str):
    """Retrieve the latest stock price for the given symbol."""
    quote = finnhub_client.quote(symbol)
    return f"The current price for {symbol} is ${quote['c']}"

# Define the tool for purchasing stock (simulated)
@tool
def purchase_stock(symbol: str, quantity: int):
    """Simulate stock purchase and return a confirmation message."""
    return f"Purchase {quantity} shares of {symbol} at the current market price."

# Register the tools in the tool node
tools = [get_stock_prices, purchase_stock]
tool_node = ToolNode(tools)

In [3]:
# Set up the AI model for reasoning
model = ChatOpenAI(model='gpt-4o-mini', api_key=keyring.get_password('openai', 'key_for_windows'))
model = model.bind_tools(tools)

In [4]:
# Define the function that simulates reasoning invokes the model
def agent_reasoning(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages":[response]}

# Define conditional logic to determine whether no continue or stop
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    # If there are no tool calls, finish the process
    if not last_message.tool_calls:
        return "end"
    return "continue"       # Otherwise, continue to the next stop

In [5]:
# Build the React agent using LangGraph
workflow = StateGraph(MessagesState)

# Add nodes: agent reasoning and tool invocation
workflow.add_node("agent_reasoning", agent_reasoning)
workflow.add_node("call_tool", tool_node)

# Define the flow
workflow.add_edge(START, "agent_reasoning")     # Start with reasoning

# Conditional edges: proceed to tool call or end the process
workflow.add_conditional_edges(
    "agent_reasoning", 
    should_continue, 
    {
        "continue":"call_tool",     # Proceed to call tool (get stock price or purchase)
        "end": END          # End the workflow
    }
)

# Normal edge : after invoking the tool, return to agent reasoning
workflow.add_edge("call_tool", "agent_reasoning")

# Set up memory for breakpoints and add 'interrupt_after' to pause after the stock purchase
memory = MemorySaver()
app = workflow.compile(checkpointer=memory, interrupt_after=["call_tool"])

# Display the graph 
display_graph(app)

d:\projects\github\learning\llm\langgraph\langgraph_blueprint\ch9/graphs/graph_67761.png


In [6]:
# Simulate user input for stock symbol
initial_input = {"messages":[{"role":"user", "content":"Should I buy AAPL stock today?"}]}
thread = {"configurable":{"thread_id":"1"}}

# run the agent reasoning step first
for event in app.stream(initial_input, thread, stream_mode="values"):
    print(event)
    
# Pausing for human approval after the purchase is made
user_approval = input("Do you approve the stock purchase action? (yes/no):")

if user_approval.lower() == "yes":
    # Confirm and finalize the transaction
    for event in app.stream(None, thread, stream_mode="values"):
        print(event)
else:
    print("Execution halted by user.")

{'messages': [HumanMessage(content='Should I buy AAPL stock today?', additional_kwargs={}, response_metadata={}, id='5d2e3d68-f7f7-4b83-84ba-1284eba99179')]}
{'messages': [HumanMessage(content='Should I buy AAPL stock today?', additional_kwargs={}, response_metadata={}, id='5d2e3d68-f7f7-4b83-84ba-1284eba99179'), AIMessage(content='To determine whether you should buy AAPL stock today, I can check the latest stock price for AAPL. Would you like me to do that?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 84, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-e7fbaef3-6557-4d16-a7b6-577290169da0-0', usage_metadata={'i